In [ ]:
import socket
import struct
import numpy as np
import matplotlib.pyplot as plt
import time

#Number of images to average 
N = 5

#initial hexapod starting location
movehexinx_initial = b"setxpos>>-4.8"
movehexiny_initial = b"setypos>>-2.5000"
movehexinz_initial = b"setzpos>>2.000"
movehexapodtooutpos = b"setypos>>20.000"

#ypos= 20 out , ypos = -2.5 in

In [ ]:
#initialisation - get beam centroid  without cap
udp_client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
hexapod_address = ('192.168.15.16', 65529)
udp_client.sendto(movehexapodtooutpos, hexapod_address)
time.sleep(30)

centroid_post = np.zeros([N,2])
#get data from probecampost
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address_probecampost = ('192.168.15.24', 65280)
client.connect(server_address_probecampost)

subcriptionstring3 = b'Wait>>MaxCounts,Centroidx,Centroidy'
SubcriptionCmdLength3 = len(subcriptionstring3)
sizepack3 = struct.pack('>i', SubcriptionCmdLength3)
client.sendall(sizepack3 + subcriptionstring3)

for x in range(N):
    size = struct.unpack('>i', client.recv(4))[0]  # Extract the msg size from four bytes - mind the encoding
    str_data = client.recv(size)
    #print('Data size: %s Data value: %s' % (size, str_data.decode('ascii')))
    centroid_post[x,0] = float(str_data.decode('ascii')[68:75])
    centroid_post[x,1] = float(str_data.decode('ascii')[101:108])
client.close()

ref_centroid_post_x = np.mean(centroid_post[:,0])
ref_centroid_post_y = np.mean(centroid_post[:,1])

centroid_postnear = np.zeros([N,2])
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address_probecampostnear = ('192.168.15.25', 65264)
client.connect(server_address_probecampostnear)

subcriptionstring4 = b'Wait>>MaxCounts,Centroidx,Centroidy'
SubcriptionCmdLength4 = len(subcriptionstring4)
sizepack4 = struct.pack('>i', SubcriptionCmdLength4)
client.sendall(sizepack4 + subcriptionstring4)

for x in range(5):
    size = struct.unpack('>i', client.recv(4))[0]  # Extract the msg size from four bytes - mind the encoding
    str_data = client.recv(size)
    #print('Data size: %s Data value: %s' % (size, str_data.decode('ascii')))
    centroid_postnear[x,0] = float(str_data.decode('ascii')[72:79])
    centroid_postnear[x,1] = float(str_data.decode('ascii')[105:112])
client.close()

ref_centroid_postnear_x = np.mean(centroid_postnear[:,0])
ref_centroid_postnear_y = np.mean(centroid_postnear[:,1])